In [26]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=32
Z_dim=100
fixed_noise=torch.randn(32,Z_dim,1,1).to(device)

class conv_block(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size, stride, padding):
        super(conv_block, self).__init__()
        
        self.conv=nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2))
        
        
    def forward(self,x):
        return self.conv(x)

    
    
    
class Discriminator(nn.Module):
    def __init__(self,in_channels, hidden_features=[64,128,256]):
        super(Discriminator, self).__init__()
        
        self.features=hidden_features
        self.layers=nn.ModuleList()
        
        #adding first conv layer in the module list it does not have a Batch normalization
        self.layers.append(
            nn.Sequential(
                nn.Conv2d(in_channels, self.features[0], kernel_size=4, stride=2, padding=1),
                nn.LeakyReLU(0.2)
                )
            )
        
        #adding hidden layers
        for i in range(len(self.features)-1):
            self.layers.append(conv_block(self.features[i],self.features[i+1], 4, 2, 1))
            
        #adding the final layer
        self.layers.append(
            nn.Sequential(
                nn.Conv2d(self.features[-1], 1, kernel_size=4, stride=2, padding=0),
                nn.Sigmoid()
                )
            )
        
        
    def forward(self,x):
        for i in self.layers:
#             print(x.shape)
            x=i(x)
        return x
    
    
# modelD=Discriminator(1)
# print(modelD)
        
    
class transpose_conv_block(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size, stride, padding):
        super(transpose_conv_block, self).__init__()
        
        self.conv=nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())
        
        
    def forward(self,x):
        return self.conv(x)
            

        
class Generator(nn.Module):
    def __init__(self,Z_dim, out_channels, hidden_features=[512,256,128]):
        super(Generator, self).__init__()
        
        self.features=hidden_features
        self.layers=nn.ModuleList()
        
        #adding first conv layer in the module list it does not have a Batch normalization
        self.layers.append(
                nn.ConvTranspose2d(Z_dim, self.features[0], kernel_size=4, stride=1, padding=0)
            )
        
        #adding hidden layers
        for i in range(len(self.features)-1):
            self.layers.append(transpose_conv_block(self.features[i],self.features[i+1], 4, 2, 1))
            
        #adding the final layer
        self.layers.append(
            nn.Sequential(
                transpose_conv_block(self.features[-1], out_channels, kernel_size=4, stride=2, padding=1),
#                 nn.Tanh()
                )
            )
        
        
    def forward(self,x):
        for i in self.layers:
#             print(x.shape)
            x=i(x)
        return x

        
# modelD=Generator(100,1)
# print(modelD)

In [27]:
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose([
#     transforms.Resize(64),
     transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# # Load the test set
# testset = torchvision.datasets.MNIST(root='./data', train=True,
#                                        download=True, transform=transform)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


In [28]:
modelG=Generator(100,3).to(device)
modelD=Discriminator(3).to(device)
optimizerG=torch.optim.Adam(modelG.parameters(), lr=2e-4, betas=(0.5,0.999))
optimizerD=torch.optim.Adam(modelD.parameters(), lr=2e-4, betas=(0.5,0.999))
lossFunction=nn.BCELoss()

def GenerateImages(fixed_noise,ep):
    fakes=modelG(fixed_noise)
    images_np = np.transpose(fakes.cpu().detach().numpy(), (0, 2, 3, 1))
    fig, axs = plt.subplots(4, 8, figsize=(20, 10))
    axs = axs.ravel()
    for i in range(images_np.shape[0]):
        axs[i].imshow(images_np[i])
        axs[i].axis('off')
    # plt.show()
    plt.savefig(f'{ep}_GI.png')
    plt.close('all')
    

def train(epoch=25):
    for ep in range(epoch):
        for i, (img,_) in enumerate(train_loader):
            #input for the discriminator
            img=img.to(device)
            # input for the generator
            noise=torch.randn(batch_size, Z_dim, 1, 1).to(device)
            
            #Generating the image from the noise
            imgG=modelG(noise)
            
            #calculating the loss for the image in the distribution
            imgD=modelD(img).reshape(-1)
            lossD=lossFunction(imgD,torch.ones_like(imgD))
            
            #calculating the loss for the generated image
            fake=modelD(imgG).reshape(-1)
            lossFake=lossFunction(fake,torch.zeros_like(fake))
            
            #optimizing the discriminator
            modelD.zero_grad()
            total_loss=(lossFake+lossD)/2
            total_loss.backward(retain_graph=True)
            optimizerD.step()
            
            #optimizing the generator
            output=modelD(imgG).reshape(-1)
            lossG=lossFunction(output,torch.ones_like(output))
            modelG.zero_grad()
            lossG.backward()
            optimizerG.step()
            
        torch.save(modelG.state_dict(), f'Generator')
        torch.save(modelD.state_dict(), f'Discriminator')
#         torch.save(modelG.state_dict(), f'{ep}_Generator')
#         torch.save(modelD.state_dict(), f'{ep}_Discriminator')
        GenerateImages(fixed_noise,ep+1)
        print(f'{ep+1} epoch Finished')

In [29]:
#generated images before training
GenerateImages(fixed_noise,0)
train(epoch=25)

1 epoch Finished
2 epoch Finished
3 epoch Finished
4 epoch Finished
5 epoch Finished
6 epoch Finished
7 epoch Finished
8 epoch Finished
9 epoch Finished
10 epoch Finished
11 epoch Finished
12 epoch Finished
13 epoch Finished
14 epoch Finished
15 epoch Finished
16 epoch Finished
17 epoch Finished
18 epoch Finished
19 epoch Finished
20 epoch Finished
21 epoch Finished
22 epoch Finished
23 epoch Finished
24 epoch Finished
25 epoch Finished


In [30]:
# To download all the generated images as a zip file execut the following command
# only if working in kaggle

from IPython.display import FileLink 

!cd "/kaggle/working/"
!zip -q GI.zip `find "/kaggle/working/" -type f -name '*_GI.png'`
print("Click below link to download the Generated images")
FileLink(r'GI.zip')

Click below link to download the Generated images


/kaggle/working/GI.zip